Definitions:

In [35]:
class State:
    state_counter = 0
    
    def __init__(self):
        self.transitions = {}
        self.id = State.state_counter
        self.is_final = False 
        State.state_counter = State.state_counter + 1


class NFA:
    def __init__(self):
        self.states = []
        self.alphabet = set()
        self.start_state = None
    
    def print(self):
        print("Alphabet:", ' '.join(sorted(self.alphabet)))
        print("States:", ' '.join([f'q{i}' for i in range(len(self.states))]))
        print("Start state: q0")
        print("Final states:", ' '.join([f'q{i}' for i, state in enumerate(self.states) if state.is_final]))
        print("Transitions:")
        for i, state in enumerate(self.states):
            for symbol, next_states in state.transitions.items():
                for next_state in next_states:
                    print(f"q{i} {symbol} q{self.states.index(next_state)}")

Parse Regex: 

In [36]:
def regex_to_nfa(regex):
    nfa = NFA()
    nfa.start_state, final_state, idx = parse_regex(regex, nfa, 0)
    final_state.is_final = True
    return nfa
    
def parse_regex(regex: str, nfa: NFA, start_idx: int):
    start = State()
    nfa.states.append(start)
    current = start
    past = None
    
    i = start_idx
    while i < len(regex):
        if regex[i].isalpha(): 
            nfa.alphabet.add(regex[i])
            next_state = State()
            nfa.states.append(next_state)
            current.transitions.setdefault(regex[i], []).append(next_state)
            past = current
            current = next_state
            
        elif regex[i] == '*' and past is not None:
            current.transitions.setdefault('ε', []).append(past)
            past.transitions.setdefault('ε', []).append(current)
        
        elif regex[i] == '|':
            next_start, next_end, j = parse_regex(regex, nfa, i + 1)
            start.transitions.setdefault('ε', []).append(next_start)
            next_state = State()
            nfa.states.append(next_state)
            current.transitions.setdefault('ε', []).append(next_state)
            next_end.transitions.setdefault('ε', []).append(next_state)
            return start, next_state, j
        
        elif regex[i] == '+' and past is not None:
            current.transitions.setdefault('ε', []).append(past)
        
        elif regex[i] == '(':
            sub_start, sub_end, j = parse_regex(regex, nfa, i + 1)
            current.transitions.setdefault('ε', []).append(sub_start)
            past = current
            current = sub_end
            i = j
            
        elif regex[i] == ')':
            return start, current, i
        
        i = i + 1
        
    return start, current, i

Validator: 

In [37]:
def validate_regex(regex: str) -> bool:
    return True

Input Section:

In [38]:
regex = input()
if not validate_regex(regex): 
    print("Invalid")
    exit(-1)
nfa = regex_to_nfa(regex)
nfa.print()

Alphabet: a b
States: q0 q1 q2 q3 q4 q5 q6 q7
Start state: q0
Final states: q7
Transitions:
q0 a q1
q1 a q2
q2 ε q3
q3 b q4
q4 a q5
q5 b q6
q6 ε q2
q6 a q7
